In [1]:
'''
This script is used for interacting with OpenLCA.
Codes are developed by Ao Chen @ETH-ML.
Connect to IPC server in OpenLCA software before running. (Port: 8080)
TODO: if you're running this script in your machine for the first time, run `product_system_reader.ipynb` first!
Search for TODO in the notebook to see how to run this notebook as needed. 
'''

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('notebook', {'highlight_selected_word': {
    'highlight_across_all_cells': True,
    'only_cells_in_scroll': False,
    'delay': 500,
    'code_cells_only': True,
}})

import os
import csv
import copy
import json
import olca
import uuid
import math
import logging
import traceback
import numpy as np
import pandas as pd
from datetime import datetime, date

from matplotlib import pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import rcParams
import matplotlib.patches as mpatches
import seaborn as sns

params = {'mathtext.default': 'regular' }

client = olca.Client(8080)
client

In [2]:
def get_all_process(client):
    # Get all process dataset in dataframe
    Process_descriptor = client.get_descriptors(olca.Process)
    Process_list = []
    ID_list = []
    Location_list = []

    for process in Process_descriptor:
        Process_list.append(process.name)
        ID_list.append(process.id)
        Location_list.append(process.location)
    Processes_df = pd.DataFrame(list(zip(Process_list,
                                    ID_list, Location_list)),
                                columns=['name', 'id', 'location'])
    return Processes_df

In [3]:
Processes_df = get_all_process(client)

In [4]:
Processes_df.head()

,name,id,location
0,"heat production, heavy fuel oil, at industrial...",4f582f90-d0ae-3552-be43-70c730e82af0,RoW
1,"containerboard production, fluting medium, rec...",fdffa34a-17fa-3676-9941-0a8a2bc5f5ac,RoW
2,"market for wood chips, dry, measured as dry ma...",35ca5293-f067-326c-ae45-920013c734c7,RER
3,"treatment of waste glass, unsanitary landfill,...",ab082ee7-7469-3a1f-8e8f-f71328b9e842,GLO
4,"heat and power co-generation, natural gas, con...",3c7c0ba3-acd4-3e19-be88-8fff9dc87788,SE


In [19]:
class LCA_Calculation():
    def __init__(self, Processes_df, Impact_method, Result_path, Calc_range_start=0, Calc_range_end=0, Excel_path='', Process_list_target=''):
        self.Processes_df = Processes_df
        self.Impact_method = Impact_method
        self.Result_path = Result_path
        self.Excel_path = Excel_path
        self.Process_list_target = Process_list_target
        self.Calc_range_start = Calc_range_start
        self.Calc_range_end = Calc_range_end

        if not os.path.exists(self.Result_path):
            os.makedirs(self.Result_path)
    
    def create_calc_product_system(self, search_process_list, Created_PS_refProcess, cached_path):
        product_system_ids = []
        product_system_names = []
        product_system_locations = []
        
        orig_json_len = len(Created_PS_refProcess)
        calc_error_csv = self.Result_path+"process_calc_error"

        for j in range(len(search_process_list)):
        # for j in range(2): # for test
            # Create product systems for selected processes
            search_process_ID = search_process_list[j]['id']
            search_process_name = search_process_list[j]['name']
            search_process_location = search_process_list[j]['location']
            if search_process_ID not in Created_PS_refProcess.keys():
                print('Creating product system of: '+search_process_name+' | '+search_process_location)
                try:
                    product_system = client.create_product_system(search_process_ID, default_providers='prefer', preferred_type='UNIT_PROCESS') #TODO: preferred_type can be changed to "SYSTEM_PROCESS" if needed
                except Exception as e:
                    print('Failed to create product system of '+search_process_ID)
                    logging.error(traceback.format_exc())
                product_system_ids.append(product_system.id)
                product_system_names.append(search_process_name)
                product_system_locations.append(search_process_location)
                Created_PS_refProcess[search_process_ID] = {'Process': [search_process_name, search_process_location, search_process_ID], 'Product_system': product_system.id}
            else:
                print('Product system of '+'"'+Created_PS_refProcess[search_process_ID]['Process'][0]+' | '+Created_PS_refProcess[search_process_ID]['Process'][1]+'"'+' has been created before.')
                print('ID: '+Created_PS_refProcess[search_process_ID]['Product_system'])
                product_system_ids.append(Created_PS_refProcess[search_process_ID]['Product_system'])
                product_system_names.append(Created_PS_refProcess[search_process_ID]['Process'][0])
                product_system_locations.append(Created_PS_refProcess[search_process_ID]['Process'][1])

        product_system_df = pd.DataFrame(list(zip(product_system_names, product_system_ids, product_system_locations)), 
                                columns=['name', 'id', 'location'])
        
        if len(Created_PS_refProcess) > orig_json_len:
            Created_PS_refProcess_file = open("Created_PS_refProcess.json", "w")
            json.dump(Created_PS_refProcess, Created_PS_refProcess_file)
            # Created_PS_refProcess_file.close()

        setup = olca.CalculationSetup()
        setup.calculation_type = olca.CalculationType.SIMPLE_CALCULATION # TODO: Can choose SIMPLE_CALCULATION, CONTRIBUTION_ANALYSIS, UPSTREAM_ANALYSIS, REGIONALIZED_CALCULATION, MONTE_CARLO_SIMULATION
        setup.impact_method = client.find(olca.ImpactMethod, self.Impact_method)
        setup.amount = 1.0
        # amount is the amount of the functional unit (fu) of the system that should be used in the calculation; unit, flow property, etc. of the fu
        # can be also defined; by default openLCA will take the settings of the reference flow of the product system

        calc_result = {}
        result_list_cache = []
        field_cache = ['GWP100a', 'location']
        result_list = []
        calc_error_list = []

        if self.Calc_range_end <= len(product_system_df):
            print("maximal number of product systems: "+str(len(product_system_df)))
            if self.Calc_range_end != 0: 
                end_point = self.Calc_range_end
                start_point = self.Calc_range_start
            else:
                end_point = len(product_system_df)
                start_point = 0
            
            cached_path_csv = cached_path+"_"+str(start_point)+"_cached.csv"
            for j in range(start_point, end_point):
                key = product_system_names[j]+' | '+product_system_locations[j]+' | '+product_system_ids[j]
                print('Calculating '+key)
                try:
                    setup.product_system = client.get(olca.ProductSystem, uid=product_system_ids[j])
                    calc_result[key] = client.calculate(setup)
                    # client.excel_export(calc_result, 'calc_result/'+product_system_locations[j]+'_'+'calc_result.xlsx')
                    result = calc_result[key]
                    if result.impact_results is not None:
                        for k in range(len(result.impact_results)):
                            GWP_result = result.impact_results[k]
                            if 'GWP 100a' in GWP_result.impact_category.name:
                                GWP100a_value = GWP_result.value
                        result_list_cache.append([GWP100a_value, product_system_locations[j]])
                        result_list.append([GWP100a_value, key])
                        with open(cached_path_csv, 'w', newline="") as f:
                            write = csv.writer(f, delimiter=";")
                            write.writerow(field_cache)
                            write.writerows(result_list_cache)
                            print('Results cached.')
                    else:
                        calc_error_list.append([product_system_names[j], product_system_locations[j], product_system_ids[j]])
                        print('No impact results: '+key)
                        with open(calc_error_csv, 'w', newline="") as f:
                            write = csv.writer(f, delimiter=";")
                            write.writerows(calc_error_list)
                            print('Error results cached.')

                except Exception as e:
                    print('Failed to calculate product system: '+product_system_ids[j])
                    print("Restart OpenLCA and Jupyter kenerl and run from No."+str(j))
                    # logging.error(traceback.format_exc())
                print('Remaining number of processes to be calculated: '+str(end_point-j-1))
        else:
            print("Input ending point exceeds maximal number of product systems: "+str(len(product_system_df)))
        try:
            result_df = pd.DataFrame(result_list, columns=['GWP100a', 'key'])
            result_df[['Activity', 'Product', 'Cutoff', 'Location', 'ps_id']] = result_df.key.str.split('|', expand=True)
            result_df = result_df.drop(columns='key')
        except Exception as e:
            result_df = ''
            print('No results are written in result_df.')
            logging.error(traceback.format_exc())

        return result_df, calc_error_list
    
    def cal_from_error_list(self, calc_error_list, Created_PS_refProcess):
        setup = olca.CalculationSetup()
        setup.calculation_type = olca.CalculationType.SIMPLE_CALCULATION # TODO: Can choose SIMPLE_CALCULATION, CONTRIBUTION_ANALYSIS, UPSTREAM_ANALYSIS, REGIONALIZED_CALCULATION, MONTE_CARLO_SIMULATION
        setup.impact_method = client.find(olca.ImpactMethod, self.Impact_method)
        setup.amount = 1.0
        result_list_cache = []
        field_cache = ['GWP100a', 'location']
        cached_path_error_csv = self.Result_path + "_calc_error_cached.csv"
        for i in range(len(calc_error_list)):
            product_system_id = calc_error_list[i][2]
            product_system_location = calc_error_list[i][1]
            for key_1, value_1 in Created_PS_refProcess.items():
                for key_2, value_2 in value_1.items():
                    if key_2 == 'Product_system':
                        if value_2 == product_system_id:
                            error_process_id = key_1
            try:
                print('Recreating product system of: '+error_process_id)
                product_system = client.create_product_system(error_process_id, default_providers='prefer', preferred_type='UNIT_PROCESS') #TODO: preferred_type can be changed to "SYSTEM_PROCESS" if needed
            except Exception as e:
                print('Failed to create product system of '+error_process_id)
                logging.error(traceback.format_exc())
                print('Try to do the calculation manually in OpenLCA.')
            Created_PS_refProcess[search_process_ID]['Product_system'] = product_system.id
            try:
                setup.product_system = client.get(olca.ProductSystem, uid=product_system.id)
                print('Recalculating: '+error_process_id)
                result = client.calculate(setup)
                if result.impact_results is not None:
                    for k in range(len(result.impact_results)):
                        GWP_result = result.impact_results[k]
                        if 'GWP 100a' in GWP_result.impact_category.name:
                            GWP100a_value = GWP_result.value
                    result_list_cache.append([GWP100a_value, product_system_location])
                    with open(cached_path_error_csv, 'w', newline="") as f:
                        write = csv.writer(f, delimiter=";")
                        write.writerow(field_cache)
                        write.writerows(result_list_cache)
                        print('Results cached.')
                else:
                    print('Still no impact results: '+product_system_id)
                    print("Try to use OpenLCA client instead.")

            except Exception as e:
                print('Still no impact results: '+product_system_id)
                print("Try to use OpenLCA client instead.")

            print('Remaining number of processes to be calculated: '+str(len(calc_error_list)-i-1))

        Created_PS_refProcess_file = open("Created_PS_refProcess.json", "w")
        json.dump(Created_PS_refProcess, Created_PS_refProcess_file)
        return Created_PS_refProcess
        
    def cal_from_excel(self, Created_PS_refProcess={}):
        result_collector = []
        if len(self.Excel_path) > 0:
            material_df = pd.read_excel(self.Excel_path, header=None)
            material_df[['Activity', 'Product', 'Cutoff']] = material_df[0].str.split('|', 2, expand=True)
            material_df['Activity'] = material_df['Activity'].str.strip()

            for i in range(len(material_df)):
            # for i in range(2): # for test
                search_process_list = [Processes_df.iloc[p] for p in range(len(self.Processes_df)) if material_df['Activity'][i] in self.Processes_df.iloc[p]['name']] # search for processes with specific name
                cached_path = self.Result_path+'/Summary_'+material_df['Activity'][i]
                result_df, calc_error_list = self.create_calc_product_system(search_process_list, Created_PS_refProcess, cached_path)
                Created_PS_refProcess = self.cal_from_error_list(calc_error_list, Created_PS_refProcess)
                if len(result_df) != 0:
                    # result_df[['GWP100a', 'Location']].to_csv(self.Result_path+'/Summary_'+material_df['Activity'][i]+'.csv', index=None, sep=";")
                    print("Results are generated in given path.")
                else:
                    print('No results are written in result_df. Check log in output.')
                result_collector.append(result_df)
            return Created_PS_refProcess, result_collector
        else:
            print('Warning: Input Excel path is not valid.')
            return Created_PS_refProcess, result_collector

    def cal_from_list(self, Created_PS_refProcess={}):
        result_collector = []
        if len(self.Process_list_target) > 0:
            for i in range(len(self.Process_list_target['include'])):
                if self.Process_list_target['not_include'][i] == '':
                    search_process_list = [Processes_df.iloc[p] for p in range(len(self.Processes_df)) if self.Process_list_target['include'][i] in self.Processes_df.iloc[p]['name']] # search for processes with specific name
                else:
                    search_process_list = [Processes_df.iloc[p] for p in range(len(self.Processes_df)) if self.Process_list_target['include'][i] in self.Processes_df.iloc[p]['name'] and not self.Process_list_target['not_include'][i] in self.Processes_df.iloc[p]['name']] # search for processes with specific name
                cached_path = self.Result_path+'/Summary_'+self.Process_list_target['include'][i]
                result_df, calc_error_list = self.create_calc_product_system(search_process_list, Created_PS_refProcess, cached_path)
                Created_PS_refProcess = self.cal_from_error_list(calc_error_list, Created_PS_refProcess)
                if len(result_df) != 0:
                    print("Results are generated in given path.")
                    # result_df[['GWP100a', 'Location']].to_csv(self.Result_path+'/Summary_'+self.Process_list_target['include'][i]+'.csv', index=None, sep=";")
                else:
                    print('No results are written in result_df. Check log in output.')
                result_collector.append(result_df)
            return Created_PS_refProcess, result_collector
        else:
            print('Warning: Process list input is not valid.')
            return Created_PS_refProcess, result_collector

In [20]:
Impact_method = 'ei -  IPCC 2013' # TODO: change impact method as you need
# TODO: if no impact results error is found, check the impact method's name

Today = date.today().strftime("%d_%b_%Y") # day, month, year
Result_path = Today+'_GWP_100a' # TODO: change result path as you need

Excel_path = ''
# Excel_path = 'List_of_Materials_ecoinvent.xlsx' # optional, 
# TODO: if no Excel would be input, Excel_path = '', 
# otherwise, change Excel name (relative path, put the excel file under the same directory as this notebook)

# Process_list_target = ''
Process_list_target = {
    'include': ['electricity, high voltage, production mix'],  # key words you want to include
    'not_include': [''] # key words you do not want to include
} # optional, 
# TODO: if no list would be input, Process_list_target = ''
# if you want to search different activities, put your keywords accordingly in the second, third, ... position 
# in the list and remember to include an empty element '' when there's no 'not_include' keyword.

Created_PS_refProcess = json.load(open("Created_PS_refProcess.json", "r")) 
# The read json file stores the processes that are calculated and their attached product system.
# So this file should be varied from machine to machine. If there're no product systems created before in 
# your OpenLCA database, the json file should contain an empty dictionary.

Calc_range_start = 80 # TODO: start point of product system to be calculated, change as needed.
Calc_range_end = 205 # TODO: end point of product system to be calculated, change as needed
instance1 = LCA_Calculation(Processes_df, Impact_method, Result_path, Calc_range_start=Calc_range_start, Calc_range_end=Calc_range_end, Excel_path=Excel_path, Process_list_target=Process_list_target)

Created_PS_refProcess, result_collector = instance1.cal_from_excel(Created_PS_refProcess=Created_PS_refProcess) 
Created_PS_refProcess, result_collector = instance1.cal_from_list(Created_PS_refProcess=Created_PS_refProcess) 

# TODO: if you see `ERROR 500`, Failed to call method public org.openlca.ipc.RpcResponse org.openlca.ipc.handlers.Calculator.calculate(org.openlca.ipc.RpcRequest)`,
# try to increase max. memory usage in OpenLCA then restart OpenLCA and Jupyter kernel.

Product system of "electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | FI" has been created before.
ID: 6b159a56-1dfa-4eb5-beac-37774ed0a2bf
Product system of "electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | IN-Eastern grid" has been created before.
ID: 2ac672c5-3e67-4352-a234-e0fa26dc8fe8
Product system of "electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | ET" has been created before.
ID: 494e5c07-8390-489f-b9a9-6f936df8f919
Product system of "electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | CN-YN" has been created before.
ID: 09b4ba3a-e3f3-423c-b572-a06312c4fad1
Product system of "electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | CN-XJ" has been created before.
ID: 70b37d21-2b10-4798-9a6d-77dcd463344f
Product system of "electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | KE" has been 

ERROR:root:calculation failed: -32000: Unhandled server error: null


No impact results: electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | AE | b9558fee-c492-45c2-88b4-91835ab9cd42
Remaining number of processes to be calculated: 107
Calculating electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | MZ | 03c91aae-139b-4248-8853-10d1a9a4f7b5
Results cached.
Remaining number of processes to be calculated: 106
Calculating electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | CN-TJ | 3e5dbae3-e5ca-49dd-adc4-67d0f1496d30
Results cached.
Remaining number of processes to be calculated: 105
Calculating electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | BR-South-eastern grid | 8f506f52-0dee-4dbf-ae07-f324b1ad34db
Results cached.
Remaining number of processes to be calculated: 104
Calculating electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | US-TRE | 68f82d24-d384-4f08-80a6-7e85f08dc849
Results cached.
Remai

In [5]:
test_json = json.load(open("Created_PS_refProcess.json", "r"))

In [11]:
test_json['781912de-560e-3ad7-9783-6cc9f5b9878f']

{'Process': ['electricity, high voltage, production mix | electricity, high voltage | Cutoff, U',
  'FI',
  '781912de-560e-3ad7-9783-6cc9f5b9878f'],
 'Product_system': '6b159a56-1dfa-4eb5-beac-37774ed0a2bf'}

In [12]:
for key_1, value_1 in test_json.items():
    for key_2, value_2 in value_1.items():
        if key_2 == 'Product_system':
            if value_2 == product_system_id:
                print(key_1)

781912de-560e-3ad7-9783-6cc9f5b9878f


In [6]:
def get_key(val):
    for key, value in test_json.items():
         if val == value:
             return key
 
    return "key doesn't exist"

# Driver Code
 
my_dict ={"java":100, "python":112, "c":11}
 
print(get_key(100))
print(get_key(11))

{'781912de-560e-3ad7-9783-6cc9f5b9878f': {'Process': ['electricity, high voltage, production mix | electricity, high voltage | Cutoff, U',
   'FI',
   '781912de-560e-3ad7-9783-6cc9f5b9878f'],
  'Product_system': '6b159a56-1dfa-4eb5-beac-37774ed0a2bf'},
 'd35f93c0-c892-35fb-b0ba-570564253b93': {'Process': ['electricity, high voltage, production mix | electricity, high voltage | Cutoff, U',
   'IN-Eastern grid',
   'd35f93c0-c892-35fb-b0ba-570564253b93'],
  'Product_system': '2ac672c5-3e67-4352-a234-e0fa26dc8fe8'},
 '01d250a6-4e98-467e-9251-b2d6f6e105f5': {'Process': ['electricity, high voltage, production mix | electricity, high voltage | Cutoff, U',
   'ET',
   '01d250a6-4e98-467e-9251-b2d6f6e105f5'],
  'Product_system': '494e5c07-8390-489f-b9a9-6f936df8f919'},
 '6341dbd7-3538-316e-873c-acb717e9ba12': {'Process': ['electricity, high voltage, production mix | electricity, high voltage | Cutoff, U',
   'CN-YN',
   '6341dbd7-3538-316e-873c-acb717e9ba12'],
  'Product_system': '09b4ba3a-e3f

In [ ]:
Impact_method = 'ei -  IPCC 2013 no LT'

In [ ]:
setup = olca.CalculationSetup()
setup.calculation_type = olca.CalculationType.SIMPLE_CALCULATION # TODO: Can choose SIMPLE_CALCULATION, CONTRIBUTION_ANALYSIS, UPSTREAM_ANALYSIS, REGIONALIZED_CALCULATION, MONTE_CARLO_SIMULATION
setup.impact_method = client.find(olca.ImpactMethod, Impact_method)
setup.amount = 1.0
setup.product_system = client.get(olca.ProductSystem, uid="6b159a56-1dfa-4eb5-beac-37774ed0a2bf")
test_result = client.calculate(setup)
# client.excel_export(calc_result, 'calc_result/'+product_system_locations[j]+'_'+'calc_result.xlsx')


In [ ]:
test_result.impact_results.value

In [ ]:
test_result.impact_results[2].value

In [7]:
# Process_list_target = ''
Process_list_target = {
    'include': ['electricity, high voltage, production mix'],  # key words you want to include
    'not_include': [''] # key words you do not want to include
} # optional, 
Created_PS_refProcess = json.load(open("Created_PS_refProcess.json", "r")) 

for i in range(len(Process_list_target['include'])):
    if Process_list_target['not_include'][i] == '':
        search_process_list = [Processes_df.iloc[p] for p in range(len(Processes_df)) if Process_list_target['include'][i] in Processes_df.iloc[p]['name']] # search for processes with specific name
product_system_ids = []
product_system_names = []
product_system_locations = []

for j in range(len(search_process_list)):
# for j in range(2): # for test
    # Create product systems for selected processes
    search_process_ID = search_process_list[j]['id']
    search_process_name = search_process_list[j]['name']
    search_process_location = search_process_list[j]['location']
    if search_process_ID not in Created_PS_refProcess.keys():
        print('Creating product system of: '+search_process_name+' | '+search_process_location)
        try:
            product_system = client.create_product_system(search_process_ID, default_providers='prefer', preferred_type='UNIT_PROCESS') #TODO: preferred_type can be changed to "SYSTEM_PROCESS" if needed
        except Exception as e:
            print('Failed to create product system of '+search_process_ID)
            logging.error(traceback.format_exc())
        product_system_ids.append(product_system.id)
        product_system_names.append(search_process_name)
        product_system_locations.append(search_process_location)
        Created_PS_refProcess[search_process_ID] = {'Process': [search_process_name, search_process_location, search_process_ID], 'Product_system': product_system.id}
    else:
        print('Product system of '+'"'+Created_PS_refProcess[search_process_ID]['Process'][0]+' | '+Created_PS_refProcess[search_process_ID]['Process'][1]+'"'+' has been created before.')
        print('ID: '+Created_PS_refProcess[search_process_ID]['Product_system'])
        product_system_ids.append(Created_PS_refProcess[search_process_ID]['Product_system'])
        product_system_names.append(Created_PS_refProcess[search_process_ID]['Process'][0])
        product_system_locations.append(Created_PS_refProcess[search_process_ID]['Process'][1])

product_system_df = pd.DataFrame(list(zip(product_system_names, product_system_ids, product_system_locations)), 
                        columns=['name', 'id', 'location'])


Product system of "electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | FI" has been created before.
ID: 6b159a56-1dfa-4eb5-beac-37774ed0a2bf
Product system of "electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | IN-Eastern grid" has been created before.
ID: 2ac672c5-3e67-4352-a234-e0fa26dc8fe8
Product system of "electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | ET" has been created before.
ID: 494e5c07-8390-489f-b9a9-6f936df8f919
Product system of "electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | CN-YN" has been created before.
ID: 09b4ba3a-e3f3-423c-b572-a06312c4fad1
Product system of "electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | CN-XJ" has been created before.
ID: 70b37d21-2b10-4798-9a6d-77dcd463344f
Product system of "electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | KE" has been 

In [8]:
len(product_system_df)

205

In [16]:
product_system_locations[80]

'CA-AB'